<h1>logistic regression without applying polynomin</h1>

do regularization with modal 'liblinear'

# <h4>score: private 0.745 - public 0.753</h4>

In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


In [2]:
dataPath = '../../preprocess_data_set/w2v_vec250.csv'


df_train = pd.read_csv(dataPath)
df_train.shape

(44898, 251)

In [3]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
0,76.089134,111.891899,83.040634,-230.633743,-58.075974,257.050781,33.574692,121.591164,1.534797,-136.893478,...,-150.614395,58.985607,202.579559,-178.152649,236.641144,413.490448,-187.805435,195.260895,-69.905609,0
1,165.381317,148.690323,112.640076,-128.012909,18.291080,83.791595,-81.372841,123.980354,11.166133,17.662382,...,-139.220505,6.865510,88.248108,-196.057449,3.397798,292.449585,-205.278229,81.772591,-107.773155,0
2,35.963501,194.378922,33.754692,-101.244164,-31.473198,58.138603,1.319836,123.404030,-97.246490,-48.950760,...,-70.667305,53.698757,159.106445,-51.326168,105.289841,116.140770,-31.169456,-9.729811,-40.059967,0
3,63.304482,127.738274,-21.535141,-59.371391,-13.245415,45.339760,3.509994,92.514626,-88.460434,-12.191431,...,-70.567566,50.938023,103.309296,-7.114745,86.394791,-0.029950,7.805309,-30.301466,-58.680027,0
4,80.099777,38.637901,27.399906,-279.661102,15.153675,213.821457,55.027672,-46.320877,-6.499963,18.798340,...,-187.330658,55.890907,201.640411,-16.693233,216.730087,100.197075,-216.734711,155.619629,-121.154930,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,71.172447,29.576962,9.814576,-114.468605,10.421734,71.369278,-19.829683,182.287994,-168.829971,-16.698744,...,-98.183441,47.153919,16.384344,7.214799,-16.159613,142.430328,-36.919342,3.927083,-52.101486,1
44894,49.767658,14.141649,24.830746,-39.896648,17.792511,-24.986475,-14.721946,48.350887,12.321180,41.165245,...,-17.996445,15.147109,20.384314,-26.027142,5.624276,43.764351,-58.034969,42.735729,-57.203762,1
44895,21.184694,193.273743,-21.810898,-881.616882,59.446217,742.972473,31.955414,826.063293,-849.959290,-8.474980,...,-1192.223511,209.601578,611.738037,44.896534,436.083008,890.661560,-1118.760498,176.455963,-366.670959,1
44896,-11.720924,-19.239174,26.855413,-27.201494,-16.071684,87.845459,19.739613,129.176987,-113.808846,3.524714,...,-71.951553,18.072672,115.359062,32.927032,64.880394,32.569496,-110.810188,89.154694,-60.245560,1


In [4]:
# randomly shuffle the rows
df_train = df_train.sample(frac=1, random_state=42)

print('Shuffled dataframe:')
df_train

Shuffled dataframe:


,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
22216,16.491493,95.172600,56.424232,-68.599670,-60.912342,7.188227,-13.449571,97.102104,-82.553505,-21.996725,...,-70.748657,19.197559,107.534119,-35.678185,11.226527,82.967758,-24.779600,17.169485,5.379145,1
27917,-72.118202,27.964672,-7.027583,10.027310,-131.827057,-7.486910,-44.090725,133.783813,-194.863373,17.914831,...,-60.451050,107.544975,108.068588,-26.898706,78.787643,99.118088,-22.362482,73.297134,-33.535873,1
25007,3.756721,-40.062057,68.558792,-42.674545,-109.676819,-14.668758,-28.305134,51.749004,-50.748463,-13.276784,...,-41.899300,57.503399,69.499634,-32.050545,-4.113337,68.741402,-36.689514,62.024269,-38.039894,1
1377,-5.599266,18.342928,7.739429,-44.109142,-22.294226,20.409508,-6.085033,5.827918,1.943156,-9.718429,...,5.241331,14.465764,17.903656,-52.080715,27.857340,58.687279,-32.958290,16.111702,3.400345,0
32476,-15.102126,-2.364689,-13.435221,-36.039989,-36.147312,-1.247078,-7.055103,24.922768,-67.551460,-39.405941,...,10.585206,34.318207,20.174820,-13.226823,-8.740134,27.602730,3.126579,42.755436,-10.994482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,60.789886,29.569677,43.278191,-118.951492,93.608200,48.588161,14.821746,34.658852,-32.712147,36.027985,...,-86.828911,18.453732,34.973248,-5.923214,41.222439,73.040588,-30.472477,45.009151,-43.001385,0
44732,-36.097504,-40.508411,1.085567,-5.865716,-61.894810,6.040255,30.072403,59.028919,-48.924541,-14.242128,...,-12.034688,33.752903,11.723240,-10.884808,-36.227303,1.101705,-45.026833,18.150423,-5.883533,1
38158,59.211033,2.794413,25.503954,-22.656900,-72.267227,-49.418575,-19.126442,35.768223,-35.190441,8.482786,...,-14.868692,-22.925610,-12.410628,-65.246872,-4.386832,60.229225,-41.607170,68.239143,-20.501524,1
860,71.339561,16.005495,51.056477,-179.901749,-14.259531,126.382141,-4.463017,42.901329,53.434349,-14.907699,...,-106.613243,-39.113091,48.542309,-67.660194,58.561333,206.557541,-90.538963,154.817764,-92.831696,0


In [5]:
df_train = df_train.reset_index()

In [6]:

df_train = df_train.drop('index', axis=1)
df_train

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
0,16.491493,95.172600,56.424232,-68.599670,-60.912342,7.188227,-13.449571,97.102104,-82.553505,-21.996725,...,-70.748657,19.197559,107.534119,-35.678185,11.226527,82.967758,-24.779600,17.169485,5.379145,1
1,-72.118202,27.964672,-7.027583,10.027310,-131.827057,-7.486910,-44.090725,133.783813,-194.863373,17.914831,...,-60.451050,107.544975,108.068588,-26.898706,78.787643,99.118088,-22.362482,73.297134,-33.535873,1
2,3.756721,-40.062057,68.558792,-42.674545,-109.676819,-14.668758,-28.305134,51.749004,-50.748463,-13.276784,...,-41.899300,57.503399,69.499634,-32.050545,-4.113337,68.741402,-36.689514,62.024269,-38.039894,1
3,-5.599266,18.342928,7.739429,-44.109142,-22.294226,20.409508,-6.085033,5.827918,1.943156,-9.718429,...,5.241331,14.465764,17.903656,-52.080715,27.857340,58.687279,-32.958290,16.111702,3.400345,0
4,-15.102126,-2.364689,-13.435221,-36.039989,-36.147312,-1.247078,-7.055103,24.922768,-67.551460,-39.405941,...,10.585206,34.318207,20.174820,-13.226823,-8.740134,27.602730,3.126579,42.755436,-10.994482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,60.789886,29.569677,43.278191,-118.951492,93.608200,48.588161,14.821746,34.658852,-32.712147,36.027985,...,-86.828911,18.453732,34.973248,-5.923214,41.222439,73.040588,-30.472477,45.009151,-43.001385,0
44894,-36.097504,-40.508411,1.085567,-5.865716,-61.894810,6.040255,30.072403,59.028919,-48.924541,-14.242128,...,-12.034688,33.752903,11.723240,-10.884808,-36.227303,1.101705,-45.026833,18.150423,-5.883533,1
44895,59.211033,2.794413,25.503954,-22.656900,-72.267227,-49.418575,-19.126442,35.768223,-35.190441,8.482786,...,-14.868692,-22.925610,-12.410628,-65.246872,-4.386832,60.229225,-41.607170,68.239143,-20.501524,1
44896,71.339561,16.005495,51.056477,-179.901749,-14.259531,126.382141,-4.463017,42.901329,53.434349,-14.907699,...,-106.613243,-39.113091,48.542309,-67.660194,58.561333,206.557541,-90.538963,154.817764,-92.831696,0


In [7]:
train_y = df_train['target']
train_x = df_train.drop(columns=['target'])


robust_scaler = RobustScaler().fit(train_x.values)
trans_x = robust_scaler.transform(train_x.values)
# trans_test = robust_scaler.transform(test.values)

In [19]:
param_grid = {"C": [.01, .1, 1], "penalty": ('l1','l2')}
logisticClassifier = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter =300, penalty = 'l1')
grid_train = GridSearchCV(logisticClassifier, param_grid, cv = 5)
grid_train.fit(trans_x, train_y)

print('best_score: ', grid_train.best_score_)

best_score:  0.9908904320424966


In [ ]:
pca = PCA(n_components = 200)
pca.fit(train_x.values)
vals_2d = pca.transform(train_x.values)

colors = ['red' if t == 1 else 'blue' for t in train_y]
for i in range(len(vals_2d)):
    plt.scatter(x = vals_2d[i,0], y = vals_2d[i,1], color = colors[i])

In [ ]:
# Create a pipeline with PCA and logistic regression
pc_num = 200

pipe = Pipeline([
    ('pca', PCA(n_components = pc_num)), # choose number of components to keep
    ('clf', LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter = 1000, penalty = 'l1'))
])

pipe.fit(trans_x, train_y)

# Create a scree plot
plt.plot(np.arange(1, pipe['pca'].n_components_+1), pipe['pca'].explained_variance_ratio_, 'bo-')
plt.xlabel('Number of PCs')
plt.ylabel('Proportion of Variance Explained')
plt.title('Scree Plot')
plt.show()

<h3>Real prediction on testing data</h3>

In [8]:
param_grid = {"C": [.01, .1], "penalty": ('l1','l2')}
logisticClassifier = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter = 300, penalty = 'l1')

# find best parameter with cross validation
grid_test = GridSearchCV(logisticClassifier, param_grid, cv=5)
grid_test.fit(trans_x, train_y)


# predictions = grid_test.predict_proba(trans_test)[:,1]

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced', max_iter=300,
                                          penalty='l1', solver='liblinear'),
             param_grid={'C': [0.01, 0.1], 'penalty': ('l1', 'l2')})

In [9]:
print("Average performance of the model: {:.3f}".format(grid_test.best_score_))

Average performance of the model: 0.988


In [10]:
grid_test.cv_results_

{'mean_fit_time': array([1.32038651, 2.83239651, 4.18224511, 5.45538301]),
 'std_fit_time': array([0.24511311, 0.30833344, 0.47889894, 1.0469824 ]),
 'mean_score_time': array([0.03791165, 0.00517826, 0.0097568 , 0.00594954]),
 'std_score_time': array([0.0618827 , 0.00146721, 0.00812102, 0.00154986]),
 'param_C': masked_array(data=[0.01, 0.01, 0.1, 0.1],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_penalty': masked_array(data=['l1', 'l2', 'l1', 'l2'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.01, 'penalty': 'l1'},
  {'C': 0.01, 'penalty': 'l2'},
  {'C': 0.1, 'penalty': 'l1'},
  {'C': 0.1, 'penalty': 'l2'}],
 'split0_test_score': array([0.97683742, 0.98207127, 0.98752784, 0.98830735]),
 'split1_test_score': array([0.97728285, 0.98374165, 0.98775056, 0.9876392 ]),
 'split2_test_score': array([0.97327394, 0.98240535, 0.98641425, 0.9876392 ]),
 'spli

In [12]:
result_df = pd.DataFrame(grid_test.cv_results_)
result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.320387,0.245113,0.037912,0.061883,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",0.976837,0.977283,0.973274,0.972714,0.976835,0.975389,0.001970,4
1,2.832397,0.308333,0.005178,0.001467,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",0.982071,0.983742,0.982405,0.980844,0.982849,0.982382,0.000952,3
2,4.182245,0.478899,0.009757,0.008121,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",0.987528,0.987751,0.986414,0.986970,0.988974,0.987527,0.000859,2
3,5.455383,1.046982,0.005950,0.001550,0.1,l2,"{'C': 0.1, 'penalty': 'l2'}",0.988307,0.987639,0.987639,0.987192,0.989531,0.988062,0.000816,1
